In [97]:
import xml.etree.ElementTree as ET
import re 

def load_xml(path):
    tree = ET.parse(path)
    return tree.getroot()
    

def load_predators(path):
    
    with open(path) as file:
        predators = file.read().splitlines()
    return predators

In [118]:
def fetch_predator_dialogs(root, predators):
    idx = 0
    pred_convos = []
    transcripts = []
    for i in root:
        convo = i.findall('message')
        authors, texts = process_convo(convo)
        if len(authors) != 2:
            continue
        a1, a2 = authors
        if a1 in predators:
            pred_convos.append(idx)
            transcript = process_transcript((a1, a2, texts))
            transcripts.append(transcript)
            continue
    
        elif a2 in predators:
            pred_convos.append(idx)
            transcript = process_transcript((a2, a1, texts))
            transcripts.append(transcript)
            continue
            
        idx+=1
    
    return transcripts

In [138]:
def process_convo(convo):
    authors = []
    texts = []
    for i in convo:
        author = i.findall('author')[0].text
        if author not in authors:
            authors.append(author)
        text = i.findall('text')[0].text
        texts.append(f'{author}: {text}')

    return authors, texts 

def process_transcript(transcript):
    
    t = '\n'.join(transcript[2])
    t=re.sub(transcript[0], 'Persuader', t)
    t=re.sub(transcript[1], 'User', t)
    return t

In [139]:
# root = load_xml(
#     '../../persuasion_datasets/pan12-sexual-predator-identification-test-and-training/' 
#     'data/pan12-sexual-predator-identification-training-corpus-2012-05-01/'
#     'pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
# )

predators = load_predators(
    '/Users/rt853/repos/UoB/persuasion_datasets/'
    'pan12-sexual-predator-identification-test-and-training/data/'
    'pan12-sexual-predator-identification-training-corpus-2012-05-01/'
    'pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt'
)

transcripts = fetch_predator_dialogs(root, predators)

In [140]:
df = pd.DataFrame(
    {'text': transcripts,
     'contains_pred' : True}
)

In [141]:
df

,text,contains_pred
0,Persuader: hey!!\nUser: hi!\nUser: how are you...,True
1,Persuader: hi\nPersuader: i guess your not on\...,True
2,Persuader: hello\nUser: hey\nPersuader: hey\nU...,True
3,User: hi\nPersuader: hi liz\nUser: what ya doi...,True
4,Persuader: Hello\nUser: hi\nPersuader: Whatcha...,True
...,...,...
1083,Persuader: hi\nUser: hey u\nPersuader: when di...,True
1084,Persuader: hey sorry had phone problems\nUser:...,True
1085,Persuader: hi\nUser: heya heya\nPersuader: wus...,True
1086,Persuader: just wanted to let you know i'm can...,True


In [142]:
import json 

def export_textfile(df, path):

    json_list = [{'text' : i, 'label' : [[]]} for i in df.text.tolist()]
    with open(path, 'w') as file:
        for i in json_list:
            file.write(json.dumps(i))
            file.write('\n')

In [143]:
export_textfile(df, '../data/testing/pred-detection-2012.jsonl')